# Practice on PCA

This practice is to get you started with the Principal Component Analysis
You will:

    - Load data from the cloud (CMIP6 historical run)
    - Format data for the PCA
    - Run PCA analysis
    - Plot results (time series and map of patterns)

Statistically, the goal is to reduce the dimensionality of the problem from $N$ grid points (this could be a collection of maps or vertical profiles) down to a few principal components.

In class, we've seen that for a (time) series (along the dimension $t$) of maps, the PCA will lead to the decomposition:
\begin{eqnarray}
	\mathbf{v}(t) &=& \sum_{j=1}^{Nc} \mathbf{P}(t,j) \mathbf{y}(j)
\end{eqnarray}
where $\mathbf{P}\in \mathbb{R}^{Nt\times Nc}$ and $\mathbf{y}\in \mathbb{R}^{Nc\times Np}$ with $Nc\leq N$. 
The first rows of $\mathbf{P}$ contain profiles maximizing the temporal variance throughout the collection of profiles.

Doc:

- https://dask-ml.readthedocs.io/en/latest/modules/generated/dask_ml.decomposition.PCA.html
- http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html    

*If you run this notebook at colab.research.google.com, you need to install packages with the following command*:

In [ ]:
!pip install --upgrade dask distributed dask-ml xarray zarr gcsfs cftime nc-time-axis intake intake-xarray scikit-learn matplotlib==3.0.2 seaborn
!apt-get -qq install python-cartopy python3-cartopy
!curl https://raw.githubusercontent.com/obidam/ds2-2020/ds2-2021/practice/exploratory_statitics/tuto_tools.py --output tuto_tools.py

## Import and set-up

In [ ]:
import numpy as np
import xarray as xr

import gcsfs
gcs = gcsfs.GCSFileSystem(token='anon') # this only needs to be created once

from dask_kubernetes import KubeCluster
from dask.distributed import Client

from tuto_tools import *

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,10)
%matplotlib inline

In [ ]:
# Keep using an existing dask cluster one:
client = Client('tcp://10.32.5.177:33497') # Update with appropriate value of the current dask cluster scheduler

In [ ]:
# or start a new one:
cluster = KubeCluster(n_workers=20)
scheduler_address = cluster.scheduler_address 
client = Client(scheduler_address) # Instantiate the dask client

In [ ]:
# Whatever your choice, you should have a client set-up:
client

# Load data to analyse

We gonna work with Atmospheric Sea Surface Pressure fields:

In [ ]:
# Look into CMIP6 data catalog:
import pandas as pd
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

In [ ]:
# Query the catalog to get the zstore entry point of a given dataset:

# Atmospheric Surface Temperature:
# df_ta = df.query("activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'tas' & experiment_id == 'historical' & source_id == 'IPSL-CM6A-LR' & member_id == 'r1i1p1f1'")

# Sea Level Pressure:
df_ta = df.query("activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'psl' & experiment_id == 'historical' & source_id == 'IPSL-CM6A-LR' & member_id == 'r1i1p1f1'")

#
df_ta

In [ ]:
# get the path to a specific zarr store (the first one from the dataframe above)
zstore = df_ta.zstore.values[-1]

# create a mutable-mapping-style interface to the store
mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
ds = xr.open_zarr(mapper, consolidated=True)
print(ds.nbytes/1e9,"Gb")
ds

In [ ]:
# We may also need the surface of cells:
df_area = df.query("variable_id == 'areacella' & source_id == 'IPSL-CM6A-LR'")
ds_area = xr.open_zarr(gcs.get_mapper(df_area.zstore.values[0]), consolidated=True)

***

1/ Create a new variable 'da' with the xarray.DataArray of interest in 'ds':

In [ ]:
da = 

2/ Get a quick look at the data (draw a map with one time slice):

3/ Create a working copy of the data array and select only the North Atlantic region:

In [ ]:
v = da.copy()
v = v.sel(lon=slice(360-100,360)).sel(lat=slice(0,90)) # North Atlantic
v

In [ ]:
# Import the PCA lib:
from dask_ml.decomposition import PCA

4/ Re-format your array to fit the PCA

To run the PCA, you need a ```[n_samples, n_features]``` array.  
You can use the xarray [```stack```](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.stack.html) method.

5/ Run the PCA with the ```fit``` method

[Click here for help on PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

In [ ]:
%%time

# Instantiate the PCA:
reducer = PCA(n_components=20, random_state=6789)

# Compute eigen vectors of the covariance matrix:


In [ ]:
# Number of components retained (the new dimensions of the reduced array)
Nc = reducer.n_components_

# Eigen vectors (maps defining the the new/reduced space):
P = reducer.components_ # [Nc , n_features], the P matrix
print(P.shape)

# Amount of variance explained by each eigen vectors (with 0 to 1 values):
eVar = reducer.explained_variance_ratio_

6/ Compute eigen values (ie the time series with intensity of each eigen vectors in this dataset):

In [ ]:
y = 

7/ Fill in new xarray DataArrays with the PCA results

The syntax to create a new xarray.DataArray is:

    xr.DataArray( <ARRAY_VALUES>, dims=<LIST_OF_DIMENSION_NAMES>, coords=<DICT_OF_DIMENSIONS>, name=<NAME_OF_THE_DataArray>)

In [ ]:
P = 
y = 

8/ Plot the annual mean time series of each PCA components:

In [ ]:
y.sel(Nc=range(0,3)).groupby('time.year').mean(dim='time').plot(hue='Nc');

9/ Plot the map of each PCA components:

In [ ]:
P.sel(Nc=slice(0,9)).plot(x='lon', y='lat', col='Nc', col_wrap=5, robust=True)

In [ ]:
10/ Plot the variance explained by each component (```eVar```):

***